# Unit test for batched augmentations

In [1]:
import torch
from datasets.datasets import SHHSdataset
import constants
import torch.utils.data as data
from datasets.augmentations import *
import matplotlib.pyplot as plt
import numpy as np


ModuleNotFoundError: No module named 'h5py'

In [3]:
ds = SHHSdataset(
    data_path=constants.SHHS_PATH_DEKSTOP,
    first_patient=1,
    num_patients=5
)
batch_size = 5
dl = data.DataLoader(dataset=ds,
                     batch_size=batch_size,
                     shuffle=False)
inputs, labels = list(iter(dl))[0]
inputs = inputs.squeeze()

## Amplitude scale


In [4]:
am = AmplitudeScale(
    mini=0.5,
    maxi=2,
    prob=1,
    batch_size=batch_size
)
am_inputs= am(inputs, inputs)
am_inputs.size()

torch.Size([5, 3000])

In [ ]:


fig, axs = plt.subplots(len(am_inputs))

for i in range(5):
    axs[i].plot(inputs[i])
    axs[i].plot(am_inputs[i])

In [5]:
inputs[0]

tensor([-0.0064, -0.0430, -0.3476,  ..., -1.3511,  0.2365, -0.3168])

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
plt.plot(np.ones([1, 10]))

In [ ]:
plt.plot([0, 1])
plt.show()

In [2]:
import matplotlib.pyplot as plt